In [2]:
import numpy as np
import pandas as pd

from bokeh.io import output_file, show
from bokeh.layouts import widgetbox, column, row
from bokeh.models.widgets import Button, RadioButtonGroup, Select, CheckboxButtonGroup
from bokeh.models import (
    GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, HoverTool,
    LogColorMapper, CustomJS, Slider, ColumnDataSource
)
from bokeh.plotting import Figure, output_file, show

In [3]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show

In [4]:
import json
with open('NYC_boundries_cleaned.json') as data_file:    
    nyc_neighborhoods = json.load(data_file)

In [5]:
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

output_file("plot.html")

palette.reverse()

neighborhood_xs = [neighborhood["Lon"] for neighborhood in nyc_neighborhoods.values()]
neighborhood_ys = [neighborhood["Lat"] for neighborhood in nyc_neighborhoods.values()]
neighborhoods = list(nyc_neighborhoods.keys())
rate = np.random.random_sample(len(neighborhoods)) 
color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=neighborhood_xs,
    y=neighborhood_ys,
    neighborhood=neighborhoods,
    rate=rate
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="NY Traffic", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color= {'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="black", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@neighborhood"),
    ("Traffic rate", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

checkbox_button_group = CheckboxButtonGroup(
        labels=["Uber", "Taxi"], active=[0, 1])
slider = Slider(start=0, end=23, step=1, title="Hour")
select1 = Select(title="Month:", value="Month", options=["April", "May", "June", "July", "August", "September"])
select2 = Select(title="Date:", value="Date", options=["1", "2", "3", "4", "5", "6", "7"])

show(row(p, checkbox_button_group, widgetbox(slider, select1, select2, width=300)))
